In [1]:
import requests
import pandas as pd
import zipfile
import io
from io import StringIO

In [24]:
pd.set_option('display.max_columns', None)

In [2]:
dic_tabelas = {
    'tabela_ncm': {
        'nome': 'ncm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv'
    },
    'tabela_sh': {
        'nome': 'sh',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_SH.csv'
    },
    'tabela_cuci': {
        'nome': 'ncm_cuci',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CUCI.csv'
    },
    'tabela_isic': {
        'nome': 'ncm_isic',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_ISIC.csv'
    },
    'tabela_isic_cuci': {
        'nome': 'isic_cuci',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/ISIC_CUCI.csv' #apagar
    },
    'tabela_cgce': {
        'nome': 'ncm_cgce',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CGCE.csv'
    },
    'tabela_fator_agregado': {
        'nome': 'ncm_fat_agreg',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_FAT_AGREG.csv'
    },
    'tabela_ppe': {
        'nome': 'ncm_ppe',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPE.csv'
    },
    'tabela_ppi': {
        'nome': 'ncm_ppi',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPI.csv'
    },
    'tabela_unidade': {
        'nome': 'ncm_unidade',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_UNIDADE.csv'
    },
    'tabela_nbm_ncm': {
        'nome': 'nbm_ncm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NBM_NCM.csv'
    },
    'tabela_nbm': {
        'nome': 'nbm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NBM.csv'
    },
    'tabela_uf': {
        'nome': 'estados',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF.csv'
    },
    'tabela_via': {
        'nome': 'via',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/VIA.csv'
    },
    'tabela_urf': {
        'nome': 'urf',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/URF.csv'
    },
    'tabela_pais': {
        'nome': 'paises',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv'
    },
    'tabela_blocos': {
        'nome': 'blocos',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS_BLOCO.csv'
    },
    'tabela_municipios': {
        'nome': 'municipios',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF_MUN.csv'
    }
}


In [3]:
def import_full(url): 
    try:
        # Requisição para obter o arquivo ZIP
        response = requests.get(url)
        response.raise_for_status()  # Levanta exceção se a resposta não for 200

        # Abrir o conteúdo como ZIP
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Lista de arquivos dentro do ZIP
            print("Arquivos no zip:", z.namelist())

            # Abrir o primeiro arquivo dentro do ZIP
            with z.open(z.namelist()[0]) as csv_file:
                # Ler o CSV para um DataFrame
                df = pd.read_csv(csv_file, sep=';', encoding='latin1') 

        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
    except zipfile.BadZipFile as e:
        print(f"Erro ao abrir o arquivo ZIP: {e}")
    except Exception as e:
        print(f"Erro ao processar o arquivo CSV: {e}")

In [ ]:

def import_tabelas_auxiliares(dic_tabelas):
    globais = globals()
    for chave, info in dic_tabelas.items():
        url = info['url']
        nome_variavel = info['nome']

        response = requests.get(url)
        response.raise_for_status()

        try:
            df = pd.read_csv(StringIO(response.text), sep=';')
        except:
            df = pd.read_csv(StringIO(response.text), sep=';')
            df = df.split()
        
        # Criar variável global com o nome especificado
        globais[nome_variavel] = df


In [5]:
fat_export = import_full('https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_COMPLETA.zip') # exportação
fat_import = import_full('https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip') # importação
import_tabelas_auxiliares(dic_tabelas)

Arquivos no zip: ['EXP_COMPLETA.csv']
Arquivos no zip: ['IMP_COMPLETA.csv']


In [19]:
fat_export = fat_export[fat_export['CO_ANO']==2025]

fat_export: 
OK:
- CO_NCM	
- CO_UNID
- CO_URF
- CO_PAIS
- CO_VIA
- QT_ESTAT
- VL_FOB - é um dos tipos de valor usados no comércio internacional.


ncm:
ok:
- CO_SH6
- CO_PPE	
- CO_PPI	
- CO_FAT_AGREG
- CO_CUCI_ITEM
- CO_CGCE_N3	
- CO_ISIC_CLASSE


In [20]:
ncm_joined = (
    ncm
    .merge(sh, left_on='CO_SH6', right_on='CO_SH6', how='inner')
    .merge(ncm_ppe, left_on='CO_PPE', right_on='CO_PPE', how='inner')
    .merge(ncm_ppi, left_on='CO_PPI', right_on='CO_PPI', how='inner')
    .merge(ncm_fat_agreg, left_on='CO_FAT_AGREG', right_on='CO_FAT_AGREG', how='inner')
    .merge(ncm_cuci, left_on='CO_CUCI_ITEM', right_on='CO_CUCI_ITEM', how='inner')
    .merge(ncm_cgce, left_on='CO_CGCE_N3', right_on='CO_CGCE_N3', how='inner')
    .merge(ncm_isic, left_on='CO_ISIC_CLASSE', right_on='CO_ISIC_CLASSE', how='inner')
)


In [21]:
fat_export_joined = (
    fat_export
    .merge(ncm_unidade, left_on='CO_UNID', right_on='CO_UNID', how='inner')
    .merge(urf, left_on='CO_URF', right_on='CO_URF', how='inner')
    .merge(paises, left_on='CO_PAIS', right_on='CO_PAIS', how='inner')
    .merge(via, left_on='CO_VIA', right_on='CO_VIA', how='inner')
)


In [22]:
final_joined = (
    fat_export_joined
    .merge(ncm_joined, left_on='CO_NCM', right_on='CO_NCM', how='inner')
)


In [25]:
final_joined

,CO_ANO,CO_MES,CO_NCM,CO_UNID_x,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,NO_UNID,SG_UNID,NO_URF,CO_PAIS_ISON3,CO_PAIS_ISOA3,NO_PAIS,NO_PAIS_ING,NO_PAIS_ESP,NO_VIA,CO_UNID_y,CO_SH6,CO_PPE,CO_PPI,CO_FAT_AGREG,CO_CUCI_ITEM,CO_CGCE_N3,CO_SIIT,CO_ISIC_CLASSE,CO_EXP_SUBSET,NO_NCM_POR,NO_NCM_ESP,NO_NCM_ING,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,CO_SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING,NO_PPE,NO_PPE_MIN,NO_PPE_ING,NO_PPI,NO_PPI_MIN,NO_PPI_ING,NO_FAT_AGREG,NO_FAT_AGREG_GP,NO_CUCI_ITEM,CO_CUCI_SUB,NO_CUCI_SUB,CO_CUCI_GRUPO,NO_CUCI_GRUPO,CO_CUCI_DIVISAO,NO_CUCI_DIVISAO,CO_CUCI_SEC,NO_CUCI_SEC,NO_CGCE_N3,NO_CGCE_N3_ING,NO_CGCE_N3_ESP,CO_CGCE_N2,NO_CGCE_N2,NO_CGCE_N2_ING,NO_CGCE_N2_ESP,CO_CGCE_N1,NO_CGCE_N1,NO_CGCE_N1_ING,NO_CGCE_N1_ESP,NO_ISIC_CLASSE,NO_ISIC_CLASSE_ING,NO_ISIC_CLASSE_ESP,CO_ISIC_GRUPO,NO_ISIC_GRUPO,NO_ISIC_GRUPO_ING,NO_ISIC_GRUPO_ESP,CO_ISIC_DIVISAO,NO_ISIC_DIVISAO,NO_ISIC_DIVISAO_ING,NO_ISIC_DIVISAO_ESP,CO_ISIC_SECAO,NO_ISIC_SECAO,NO_ISIC_SECAO_ING,NO_ISIC_SECAO_ESP
0,2025,1,4014021,10,580,PR,1,917800,2,2,23,QUILOGRAMA LIQUIDO,KGL,0917800 - PORTO DE PARANAGUA,591,PAN,Panamá,Panama,Panamá,MARITIMA,10,40140,3990.0,3198.0,3.0,02213,323,9000.0,1050,2200.0,"Creme de leite UHT (Ultra High Temperature), c...","Cremas de leche uht, grasa > 6, pero <= 10% de...","Milk cream uht, fat > 6º, but <= 10% of weight","Leite e creme de leite, com teor em peso, de m...",Leche y nata con un contenido en peso de grasa...,"Milk and cream of a fat content, by weight, ex...",401,"Leite e nata, não concentrados nem adicionados...","Leche y nata (crema), sin concentrar, sin adic...","Milk and cream, not concentrated nor containin...",4,Leite e lacticínios; ovos de aves; mel natural...,Leche y productos lácteos; huevos de ave; miel...,Dairy produce; birds' eggs; natural honey; others,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products,DEMAIS PRODUTOS MANUFATURADOS,Demais produtos manufaturados,OTHER MANUFACTURED GOODS,"PRODUTOS CONSTITUIDOS DO LEITE, OUTROS","Produtos constituídos do leite, outros","PRODUCTS CONTAINING MILK, OTHER",PRODUTOS MANUFATURADOS,PRODUTOS INDUSTRIALIZADOS,Creme com conteúdo de gordura excedendo a 6% d...,221,"Leite (incluindo desnatado) e creme, não conce...",022,"Leite, creme de leite e laticínios, exceto man...",2,Laticínios e ovos de aves,0,PRODUTOS ALIMENTICIOS E ANIMAIS VIVOS,"Alimentos e bebidas básicos, destinados princi...","Basic food and beverages, mainly for domestic ...","Alimentos y bebidas básicos, principalmente pa...",32,Bens de consumo semiduráveis e não duráveis,Consumer goods semi-durable and non-durable,Bienes de consumo semidurables y no durables,3,BENS DE CONSUMO (BC),CONSUMER GOODS (BC),BIENES DE CONSUMO (BC),Fabricação de produtos lácteos,Manufacture of dairy products,Fabricação de produtos lácteos,105,Fabricação de produtos lácteos,Manufacture of dairy products,Fabricação de produtos lácteos,10,Fabricação de produtos alimentícios,Manufacture of food products,Fabricação de produtos alimentícios,C,Indústria de Transformação,Manufacturing,Indústria de Transformação
1,2025,3,7061000,10,476,RJ,1,717800,477,477,912,QUILOGRAMA LIQUIDO,KGL,0717800 - PORTO DE ITAGUAI,584,MHL,"Marshall, Ilhas",Marshall Islands,Islas Marshall,MARITIMA,10,70610,1990.0,1990.0,1.0,05455,323,9000.0,113,8099.0,"Cenouras e nabos, frescos ou refrigerados","Zanahorias y nabos, frescos o refrigerados",Fresh/chilled carrots and turnips,"Cenouras e nabos, frescos ou refrigerados","Zanahorias y nabos, frescos o refrigerados (ex...",Fresh or chilled carrots and turnips,706,"Cenouras, nabos, beterrabas para salada, cerce...","Zanahorias, nabos, remolachas para ensalada, s...","Carrots, turnips, salad beetroot, salsify, cel...",7,"Produtos hortícolas, plantas, raízes e tubércu...","Hortalizas, plantas, raíces y tubérculos alime...",Edible vegetables and certain roots and tubers,

In [ ]:
#final_joined = (
#    fat_export_joined
#    .merge(ncm_joined, left_on='CO_NCM', right_on='CO_NCM', how='inner')
#)